In [15]:
from google.colab import files

from scipy import optimize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)

In [2]:
# upload file and read it in
dataset = files.upload()
df = pd.read_csv('Exits.csv', delimiter=',', index_col=False)

Saving Exits.csv to Exits.csv


In [19]:
potentialRRs = np.around(df["Potential RR"].to_numpy(), 1)
potentialRRs = np.unique(potentialRRs)

totalPotentialRR = np.sum(potentialRRs)
largestRR = np.max(potentialRRs)

(5432,)
(161,)


In [20]:
# calculate RR earned based on partials
def earned_rr(partials):
    p1 = partials[0]
    p1_percent = partials[1]
    p2 = partials[2]
    p2_percent = partials[3]

    penality = -10000
    if p1_percent + p2_percent > 1:
      return penality

    # if p1 > p2:
    #   return penality
    
    earned_rr = 0
    
    for rr in potentialRRs:
        if rr < p1: 
            earned_rr -= 1
        elif (rr >= p1 and rr <= p2) or (p2_percent == 0):
            earned_rr += p1*p1_percent
        elif rr >= p2:
            earned_rr += p2*p1_percent*p2_percent
    
    return earned_rr

# the closer we are to the toal RR possible, the better
def earned_rr_loss(partials):   
    return totalPotentialRR - earned_rr(partials)

In [21]:
columns = ['Earned RR', 'First Partial Value', 'First Partial %', 'Second Partial Value', 'Second Partial %']
rrs = pd.DataFrame(columns=columns)

In [22]:
# limit partial percentages to equal 1 since we can't take off more than 100% of a position
cons = [
    {'type': 'eq', 'fun': lambda x: x[1] + x[3] - 1 },
    {'type': 'ineq', 'fun': lambda x: x[0] - x[2] }
]

In [23]:
%%capture

rrLineSpace = np.linspace(0, largestRR, 10, dtype=int)
for partialOne in potentialRRs:
  for partialTwo in potentialRRs:
    optimized = optimize.minimize(earned_rr_loss, [partialOne, 0.5, partialTwo, 0.5], 
                            bounds=((0, largestRR), (0, 1.), (0, largestRR), (0, 1.)), method='Nelder-Mead', constraints=cons)
    
    rrs = rrs.append(
    {
      'Earned RR': earned_rr(optimized.x), 
      'First Partial Value': optimized.x[0], 
      'First Partial %': optimized.x[1], 
      'Second Partial Value': optimized.x[2], 
      'Second Partial %': optimized.x[3]
      }, ignore_index=True)

In [24]:
# values may differ slighly per run based on how well the optimizer finds a minimum but it shouldn't matter if you round 
max_rr_row = rrs.iloc[rrs['Earned RR'].idxmax()]
print(f'''The most "Total RR" achieved is {max_rr_row['Earned RR']:.3f} by takning {max_rr_row['First Partial %']:.2f}% off at {max_rr_row['First Partial Value']:.3f}
      RR and then {max_rr_row['Second Partial %']:.2f}% off at {max_rr_row['Second Partial Value']:.3f} RR''')

The most "Total RR" achieved is 544.600 by takning 1.00% off at 7.400
      RR and then 0.00% off at 17.236 RR
